<a href="https://colab.research.google.com/github/luminyanko/DSL/blob/main/job2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job #2 
Savenko Valeriia

Let's say that context free grammar is such this:
```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)
class : int
value : str
var : str                 # non-terminal name
definition : list(rule)
rule : list(var | token)  # right part of the rule
```

Non-terminals and corresponding rules are passed in a cycle. If the non-terminal is not foreign, it remains and is added to the new dictionary (it is checked whether such does not already exist in the dictionary), if it is foreign - it is ignored. Rules that do not correspond to non-third-party non-terminals are also moved: if the rule contains a third-party non-terminal, it is rejected.

In [25]:
def remove_non_terminal(grammar):
  not_existing_sym = search_not_existing_sym(grammar)
  new_grammar = copy.deepcopy(grammar)
  toks = grammar['toks']
  vars = grammar['vars']
  new_vars = dict()

  for nonterminals, definitions in vars.items():
    if nonterminals in not_existing_sym:
             for rules in definitions:
                 if check_token_for_rule(rules, toks, not_existing_sym):
                    if nonterminals in new_vars.keys():
                        new_vars[nonterminals].append(rules)
                    else:
                        new_vars[nonterminals] = list()
                        new_vars[nonterminals].append(rules)
    

Non-third-party terminals are defined by an auxiliary function that checks whether the non-terminal has only tokens and / or already known non-third-party non-terminals in the rule.

In [22]:
def search_not_existing_sym(grammar):
    toks = grammar['toks']
    vars = grammar['vars']
    not_existing_sym = list()
    flag = True
    while flag:
        flag = False
        for nonterminals, definitions in vars.items():
            for rule in definitions:
                if check_token_for_rule(rule, toks, not_existing_sym):
                    if nonterminals not in not_existing_sym:
                        not_existing_sym.append(nonterminals)
                        flag = True
    return not_existing_sym

Function that checks the rules:

In [16]:
def check_token_for_rule(rule, tokens, _list):
    for partOfRule in rule:
        if partOfRule in tokens or partOfRule in _list:
            pass
        else:
            return False
    return True

Here's an examlple: 

In [34]:
import copy
import pprint
grammar = { 'toks': list(
                    [(0, ' '),
                     (11, 'aa'),
                     (12, 'bb'),
                     (21, 'ab'),
                     (22, 'cc')]
                    ),
            'vars': {
                'A': [['B', (11, 'aa')],
                      ['D'],
                      ['F', (12, 'bb')]],
                'B': [['B'],
                      ['F', (21, 'ab')]],
                'C': [['C'],
                      ['A',(22, 'cc')],
                      ['F', (11, 'aa')]],
                'D': [[(0, ' ')]],
                'F': [
                      ['B', (21, 'ab')],
                      ['F', (12, 'bb')]]
                    },
            'hvar': 'A',
            }
remove_non_terminal(grammar)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(grammar)

{   'hvar': 'A',
    'toks': [(0, ' '), (11, 'aa'), (12, 'bb'), (21, 'ab'), (22, 'cc')],
    'vars': {   'A': [['B', (11, 'aa')], ['D'], ['F', (12, 'bb')]],
                'B': [['B'], ['F', (21, 'ab')]],
                'C': [['C'], ['A', (22, 'cc')], ['F', (11, 'aa')]],
                'D': [[(0, ' ')]],
                'F': [['B', (21, 'ab')], ['F', (12, 'bb')]]}}
